In [2]:
"""
Sanity check that our use of cv2.undistoryPoints is correct.
The alternative is to undistort the image first and then find a point in the undistorted image, but this is
computationally inefficient.
"""

import cv2
from im_proc import select_test_correspondences, get_brightest_contour
from stereo_calibration import *
import pickle
import numpy as np

with open("params/stereo_params_jul3.pkl", "rb") as f:
    stereo_params = pickle.load(f)

left_img = cv2.imread("img1.png")
left_blob = get_brightest_contour(left_img, otsu_adjustment=20)
right_img = cv2.imread("img2.png")
right_blob = get_brightest_contour(right_img, otsu_adjustment=15)

# find blobs in distorted images and then undistort the blob center points
uvs1 = np.array([left_blob["center"]]).astype(np.float32)
uvs2 = np.array([right_blob["center"]]).astype(np.float32)
uvs1_undistorted = cv2.undistortPoints(uvs1, stereo_params.cam1_params.K, stereo_params.cam1_params.D, P = stereo_params.cam1_params.K)
uvs2_undistorted = cv2.undistortPoints(uvs2, stereo_params.cam2_params.K, stereo_params.cam2_params.D, P = stereo_params.cam2_params.K)
uvs1_undistorted = uvs1_undistorted.astype(int)
uvs2_undistorted = uvs2_undistorted.astype(int)
left_img_vis = left_img.copy()
right_img_vis = right_img.copy()
cv2.drawMarker(left_img_vis, uvs1_undistorted[0][0], (0, 255, 0), markerSize=30, thickness=2)
cv2.drawMarker(right_img_vis, uvs2_undistorted[0][0], (0, 255, 0), markerSize=30, thickness=2)
print(uvs1_undistorted[0][0], uvs2_undistorted[0][0])
disp_img = np.hstack((left_img_vis, right_img_vis))
cv2.imshow("blobs", disp_img)
k = cv2.waitKey(1)
if k == ord('q'):
    cv2.destroyAllWindows()



# now undistort image first and then find the blob
left_img_undistorted = undistort(left_img, stereo_params.cam1_params)
right_img_undistorted = undistort(right_img, stereo_params.cam2_params)
left_blob = get_brightest_contour(left_img_undistorted, otsu_adjustment=20)
right_blob = get_brightest_contour(right_img_undistorted, otsu_adjustment=15)
cv2.drawMarker(left_img_undistorted, tuple(left_blob["center"]), (0, 0, 255), markerSize=30, thickness=2)
cv2.drawMarker(right_img_undistorted, tuple(right_blob["center"]), (0, 0, 255), markerSize=30, thickness=2)
cv2.drawMarker(left_img_undistorted, np.array(uvs1_undistorted[0][0]).astype(int), (0, 255, 0), markerSize=30, thickness=2)
cv2.drawMarker(right_img_undistorted, np.array(uvs2_undistorted[0][0]).astype(int), (0, 255, 0), markerSize=30, thickness=2)
print(left_blob["center"], right_blob["center"])
undistorted_disp_img = np.hstack((left_img_undistorted, right_img_undistorted))
cv2.imshow("Undistorted", undistorted_disp_img)
k = cv2.waitKey(1)
if k == ord('q'):
    cv2.destroyAllWindows()



[681 338] [872 197]
(682, 339) (872, 196)


: 